In [1]:
## Imports
import numpy as np
import pandas as pd
import os
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from scipy.stats import pearsonr

from math import sqrt

current_directory = os.getcwd()
print(current_directory)

d:\OneDrive\ML\Model


In [2]:
data = pd.read_excel("CSVs/ML_Inputs/ML_Training_Numerical_MaxMoment.xlsx",header=None)
data = data.drop(data.columns[0], axis=1)

data.columns = data.iloc[0]
data=data.drop(0)


In [3]:
## Predicting VAriable
drop_columns=['inj_rate_max','inj_depth_max','inj_depth_min','moment_max','rock_biot_min']
pred_variable = 'moment_max'

X = data.drop(columns=drop_columns)
y = data[pred_variable]

X_train = X[y.notnull()]
y_train = y[y.notnull()]

X_pred = X[y.isnull()]

X_train_split, X_test_split, y_train_split, y_test_split = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

model = RandomForestRegressor(n_estimators=100, random_state=42)

model.fit(X_train_split, y_train_split)

y_pred_split = model.predict(X_test_split)
rmse = sqrt(mean_squared_error(y_test_split, y_pred_split))
print(f"Model RMSE: {rmse}")
X_pred = X_pred.apply(pd.to_numeric, errors='coerce')



if not X_pred.empty:
    predicted_biot = model.predict(X_pred)
    data.loc[data[pred_variable].isnull(), pred_variable] = predicted_biot

feature_importances = pd.DataFrame(
    {"Feature": X.columns, "Importance": model.feature_importances_}
)
feature_importances.to_excel(f"CSVs/ML_Outputs/feature_importances_{pred_variable}.xlsx", index=False)


# Save the updated dataset to a new file
data.to_excel(f"CSVs/ML_Outputs/ML_predicted_{pred_variable}.xlsx", index=False)




Model RMSE: 2.1727055361703536


In [4]:
drop_columns=['moment_max']
pred_variable = 'moment_max'
X = data.drop(columns=drop_columns)
y = data[pred_variable]

X_train = X[y.notnull()]
y_train = y[y.notnull()]

X_pred = X[y.isnull()]

X_train_split, X_test_split, y_train_split, y_test_split = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

model = RandomForestRegressor(n_estimators=100, random_state=42)

model.fit(X_train_split, y_train_split)

y_pred_split = model.predict(X_test_split)
rmse = sqrt(mean_squared_error(y_test_split, y_pred_split))
print(f"Model RMSE: {rmse}")
X_pred = X_pred.apply(pd.to_numeric, errors='coerce')


if not X_pred.empty:
    predicted_var = model.predict(X_pred)
    data.loc[data[pred_variable].isnull(), pred_variable] = predicted_var

feature_importances = pd.DataFrame(
    {"Feature": X.columns, "Importance": model.feature_importances_}
)
feature_importances.to_excel(f"CSVs/ML_Outputs/feature_importances_{pred_variable}.xlsx", index=False)
# Save the updated dataset to a new file
data.to_excel(f"CSVs/ML_Outputs/ML_predicted_{pred_variable}.xlsx", index=False)

Model RMSE: 2.5015153836998514


In [5]:
### Predicting together
X = data.drop(columns=['rock_biot_min', 'rock_biot_max'])
y = data[['rock_biot_min', 'rock_biot_max']]

X = X.apply(pd.to_numeric, errors='coerce') 
y = y.apply(pd.to_numeric, errors='coerce') 

X_train = X[y.notnull().any(axis=1)]  
y_train = y[y.notnull().any(axis=1)]

X_pred = X[y.isnull().any(axis=1)]  


X_train_split, X_test_split, y_train_split, y_test_split = train_test_split(X_train, y_train, test_size=0.2, random_state=42)


model = RandomForestRegressor(n_estimators=100, random_state=42)


model.fit(X_train_split, y_train_split)


y_pred_split = model.predict(X_test_split)


rmse_min = sqrt(mean_squared_error(y_test_split['rock_biot_min'], y_pred_split[:, 0]))
rmse_max = sqrt(mean_squared_error(y_test_split['rock_biot_max'], y_pred_split[:, 1]))

print(f"Model RMSE for rock_biot_min: {rmse_min}")
print(f"Model RMSE for rock_biot_max: {rmse_max}")


X_pred = X_pred.apply(pd.to_numeric, errors='coerce')

if not X_pred.empty:
    predicted_biot = model.predict(X_pred)
    
    data.loc[data["rock_biot_min"].isnull(), "rock_biot_min"] = predicted_biot[:, 0]
    data.loc[data["rock_biot_max"].isnull(), "rock_biot_max"] = predicted_biot[:, 1]

# Extract feature importances
feature_importances = pd.DataFrame(
    {"Feature": X.columns, "Importance": model.feature_importances_}
)


feature_importances.to_excel("CSVs/ML_Outputs/feature_importances_biot.xlsx", index=False)


data.to_excel("CSVs/ML_Outputs/ML_predicted_biot.xlsx", index=False)


Model RMSE for rock_biot_min: 0.04641385386699611
Model RMSE for rock_biot_max: 0.06390033010747108
